# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

## Get data

### Import CIFAR10 data

In [0]:
from tensorflow.keras.datasets import cifar10

### Get train and test data
Use these variable names

X_train, X_test, y_train, y_test

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Print the shape of X_train and y_train

In [0]:
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)

### Rehshape y_train and y_test
reshape with y_train.shape[0] and y_test.shape[0]

In [0]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

### Create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
X_train_lt5 = X_train[y_train<5]
X_test_lt5 = X_test[y_test<5]
y_train_lt5 = y_train[y_train<5]
y_test_lt5 = y_test[y_test<5]

X_train_gt5 = X_train[y_train>=5]
X_test_gt5 = X_test[y_test>=5]
y_train_gt5 = y_train[y_train>=5]
y_test_gt5 = y_test[y_test>=5]

## Use One-hot encoding to divide y_train and y_test into required no of output classes
Do it for both datasets

In [0]:
from pandas import get_dummies
y_train_lt5 = get_dummies(y_train_lt5)
y_test_lt5 = get_dummies(y_test_lt5)
y_train_gt5 = get_dummies(y_train_gt5)
y_test_gt5 = get_dummies(y_test_gt5)

### Print data variables for dataset having classes from 0 to 4
Data variables here are referring to X-train, X_test, y_train, y_test of that dataset

In [0]:
print(X_train_lt5.shape)
print(X_test_lt5.shape)
print(y_train_lt5.shape)
print(y_test_lt5.shape)

### Print data variables for dataset having classes from 5 to 9
Data variables here are referring to X-train, X_test, y_train, y_test of that dataset

In [0]:
print(X_train_gt5.shape)
print(X_test_gt5.shape)
print(y_train_gt5.shape)
print(y_test_gt5.shape)

## Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

### Initialize a model and add the required layers

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(32,32,3)))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(BatchNormalization())

model.add(Dense(units=128, activation='relu'))

model.add(BatchNormalization())

model.add(Dense(units=64, activation='relu'))

model.add(Dense(units=5, activation='softmax'))

### Summarize your model

In [0]:
model.summary()

### Compile and fit the model

In [0]:
from tensorflow.keras.optimizers import Adam
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.01)
model.fit(X_train_lt5, y_train_lt5, validation_data=(X_test_lt5, y_test_lt5), epochs=10, batch_size=32, callbacks=callback)

### Evaluate your model

In [0]:
model.evaluate(X_test_lt5, y_test_lt5)

## In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for layers in model.layers:
  if('dense' not in layers.name):
    layers.trainable = False

### Print in colors

In [0]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

## Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_gt5, y_train_gt5, validation_data = (x_test_gt5, y_test_gt5), epochs = 5, batch_size = 32)

In [0]:
model.evaluate(x_test_gt5, y_test_gt5)